In [1]:
import sys
from pathlib import Path
import os 

PROJECT_ROOT = Path.cwd().parents[0]   # adjust if needed
sys.path.append(str(PROJECT_ROOT))

from src.metrics import cross_spectrum_irregular, auto_spectrum_irregular, integrate_cross_spectrum_real
from src.read_data import read_ogle_dat
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
rng = np.random.default_rng(0)
t = np.sort(rng.uniform(0, 30, 500))
y_syn = np.sin(2*np.pi*0.7*t) + 0.05*rng.normal(size=t.size)

delta_f = 0.001
delta_t = (t.max() - t.min()) / len(t)
f_min_syn = delta_f
f_max_syn = 1/(2*delta_t)

Iy_syn = integrate_cross_spectrum_real(f_min_syn, f_max_syn, t, y_syn, t, y_syn, jacobian="df")
print("Iy_syn =", Iy_syn)

Iy_syn = 17.107177304126896


c:\Users\fperezg\Desktop\papers\period estimation Light curves\code\Comparing_irregular_sampled_time_series_using_JDR\Python files\JDR-python\src\metrics.py:375: IntegrationWarning: The maximum number of subdivisions (200) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  value, _err = quad(integrand, f_lower, f_upper, limit=quad_limit, epsabs=1e-10, epsrel=1e-10)


In [3]:
f_grid = np.linspace(f_min_syn, f_max_syn, 2000)
omega_grid = 2*np.pi*f_grid

Syy = auto_spectrum_irregular(omega_grid, t, y_syn)

print("max|Im| =", np.max(np.abs(np.imag(Syy))))
print("max|Re| =", np.max(np.abs(np.real(Syy))))


max|Im| = 0.0
max|Re| = 257.6369732490742


In [4]:
Iy_trap = np.trapz(np.real(Syy), f_grid)
print("Iy_quad =", Iy_syn)
print("Iy_trap =", Iy_trap)
print("rel diff =", abs(Iy_syn - Iy_trap)/max(1e-12, abs(Iy_trap)))


Iy_quad = 17.107177304126896
Iy_trap = 17.151080843013958
rel diff = 0.002559811786144335


C:\Users\fperezg\AppData\Local\Temp\ipykernel_21892\127250039.py:1: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  Iy_trap = np.trapz(np.real(Syy), f_grid)


In [5]:
from scipy.integrate import quad
import numpy as np

def Iy_strict(f_min, f_max, t, y):
    def integrand(f):
        w = 2*np.pi*f
        return float(np.real(cross_spectrum_irregular(w, t, y, t, y)))
    val, _ = quad(integrand, f_min, f_max, limit=800, epsabs=1e-10, epsrel=1e-10)
    return float(val)

print("Iy_strict =", Iy_strict(f_min_syn, f_max_syn, t, y_syn))


Iy_strict = 17.115210810075688


C:\Users\fperezg\AppData\Local\Temp\ipykernel_21892\3950424404.py:8: IntegrationWarning: The maximum number of subdivisions (800) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  val, _ = quad(integrand, f_min, f_max, limit=800, epsabs=1e-10, epsrel=1e-10)
